!pip install langchain==0.0.165

# base LLM

https://python.langchain.com/en/latest/modules/models/llms/examples/custom_llm.html

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

C:\Users\jimwa\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

In [12]:
class CustomLLM(LLM):
    
      
    @property
    def _llm_type(self) -> str:
        return "custom"

    def prompt_to_resoonse(
        self,
        prompt,
        max_length = 128,
        ):
        inputs = tokenizer(
            prompt, 
            return_tensors="pt",
            )
        model_output = model.generate(
            **inputs,
            output_scores=True,
            max_length = max_length,
            )
        response = tokenizer.batch_decode(
            model_output, 
            skip_special_tokens=True)[0]
        return response
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        response = self.prompt_to_resoonse(prompt)
        return response
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {}

In [15]:
llm = CustomLLM()
llm("My name is Jim. Question: What is my name?")

'Jim'

# prompt template

In [18]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [19]:
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


In [20]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [26]:
chain.run("colorful socks")

'samsung'

# momory

In [35]:
from langchain import OpenAI, ConversationChain

conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
Hi, I'm sorry to hear that. I'm sorry to hear that.
